### Embeddings
这里不但包括了单词的embeddings，同时还包括了各种其他的属性的embedding和position的。<br>
不同feature之间存在三种融合的方式，即concat，sum，mlp。<br>
模型是由多个nn.Embedding模型(针对不同的features)组合而成，根据输入和参数的不同，呈现不同数目及大小的模型。<br>
输出的embedding已经融合了所要求的features
#### 属性
- self.positional_encoding: Boolean, action = 'store_true'是否使用positional_encoding.
- self.pe: [max_len, 1, word_vec_len] max_len是最大可能的位置数量。保存的代表不同位置的向量。使向量长度为词向量的长度，是为了方便和单词的词向量进行结合。
- self.feat_merge: choices=['concat', 'sum', 'mlp']。指定不同features embedding之间的融合方式
- feat_exp = opt.feat_vec_exponent: 指定如果merge方式使用concat的话，融合后的向量长度，应该为对应feature的值域空间的feat_exp次方。default = 0.7
- vocab_sizes：列表，指各种属性的值域空间的大小。
- emb_sizes: 列表，指各种属性生成的向量的长度大小。
- self.mlp: 当使用mlp融合时，mlp的结构
- self.emb_luts: nn.ModuleList保存各个feature的embedding模型。

#### 方法
- __init__(self, opt, dicts, feature_dicts = None): 设置dict，feature dict的embedding模型。
- word_lut(self): 输出单词的embedding模型
- embedding_size(self): 返回词向量的长度
- make_positional_encoding(self, dim, mac_len): 生成positional embedding
- load_pretrained_vectors(self, emb_file): 顾名思义
- merge(self, features): 融合
- forward(self, src_input_): 运行方法

### Encoder
设定MT系统的Encoder部分，可以调节层的数量及每个层中节点的数量以及节点的类型。模型输入由前置的embedding模型得到。<br>
目前支持三种类型的层：即GRU，LSTM和Transformer。<br>
#### 属性
- self.layers: int, encoder层的个数
- self.num_directions: int [0, 1] 是否bidirection
- self.hidden_size: 每个隐藏层的节点的数量，这里指的是每个方向的数量，如果是bidirection，那么总的数量应该是它的两倍。
- self.embeddings：embedding模型
- self.encoder_layer: choices = ['GRU', 'LSTM', 'Transformer'] 指定每个层的用的结构类型
- self.transformer: 如果encoder_layer是Transformer，设定为对应的transformer模型（onmt.modules.TransformerEncoder）列表（每一项代表一层）
- self.rnn: 如果encoder_layer不是Transformer，设定为对应的rnn模型（nn.LSTM/nn.GRU）列表

#### assert
assert opt.rnn_size % self.num_direction == 0

#### 方法
__init__(self, opt, dicts, feature_dicts = None)
forward(self, intput, lengths = None, hidden = None): length是指batch_size，hidden是隐藏层的初始值。

#### 图
还木画

### Decoder
设定MT模型的Decoder部分，可以设定层的数量及每个层中节点的数量以及节点的类型，attention的类型。<br>
目前支持的层的类型有：GRU，LSTM，Transformer<br>
目前支持的attention的类型有：dot, general, mlp, copy<br>
目前支持的结合context的方式有：source, target, both<br>

#### 属性
- self.layers: int, decoder层的个数，encoder和decoder层数相同
- self.decoder_layer: int, decoder每个层的类型
- self._coverage: action='store_true' 指定attention是否要使用coverage。用额外的向量来记录已经翻译到什么地方，从而实现coverage。
- self.hidden_size: 因为不用考虑bidrectional所以等于opt.rnn_size
- self.input_feed: 指定是否使用额外的context信息作为输入信息，如果是将扩充input_size(原始为word_vec_size)，default = 1
- self.transformer: 如果decoder_layer是Transformer，设定为对应的Transformer模型（onmt.modules.TransformerDecoder()）列表
- self.rnn: 如果decoder_layer不是Transformer，设定为对应的rnn模型(onmt.modules.StackedLSTM/GRU)列表
- self.context_gate: 指定使用的context_gate的类型，choices = ['source', 'target', 'both']。当使用input_feed时，context_gate作为权值，权衡contxt和原始input之间的比重
- self.dropout: dropput模型。
- self.attn: attention模型，可以指定attention的类型，choices = ['dot', 'general', 'mlp']
- self._copy: 指定是否训练copy attention模型（onmt.modules.GlobalAttention）， action = 'store_true'

#### 方法
- __init__(self, opt, dicts)<br>
- forward(self, input, src, context, state): context [src_len, batch, rnn_size], state是一个用于初始化decode的对象

#### 图
还木画

### NMTModel

#### 属性
- self.multigpu: boolean是否使用multi gpu
- self.encoder
- self.decoder

#### 方法
- __init__(self, encoder, decoder, multigpu = False)
- _fix_enc_hidden(sellf, h): change [layers*directions, batch, dim] to [layers, batch, directions*dim]
- init_decoder_state(self, context, enc_hidden)
- forward(self, src, tgt, lengths, dec_state = None)

### RNNDecoderState(DecoderState)

#### 属性
self.hidden: rnn中的可训练变量[]

dict features不是很了解，有什么features

In [3]:
a = [5]


### 问题
copy attention并不知道是什么东西